In [1]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from pyproj import Proj
import os,shutil,random
import numpy as np
import xarray as xr
from itertools import chain

def plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,nx,ny,lat_0,lon_0):

#     m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='cyl', ax=ax)   
    m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='tmerc', ax=ax,lat_0=lat_0,lon_0=lon_0)

    m.drawmapboundary(color='k', linewidth=1.)
    m.drawcoastlines()
   
    # lat and lon with lables
    m.drawparallels(np.arange(np.floor(llcrnrlat),np.ceil(urcrnrlat),0.2),labels=[True,False,False,False],dashes=[1,1], fontsize='small') # Draw parallels (latitude lines) for values (in degrees).
    m.drawmeridians(np.arange(np.floor(llcrnrlon),np.ceil(urcrnrlon),0.2),labels=[False,False,False,True],dashes=[1,1], fontsize='small') # Draw meridians (longitude lines). Label [left, right, top, bottom]

    # draw a shaded-relief image
    m.shadedrelief(scale=0.5)
    
    # lats and longs are returned as a dictionary
#     lats = m.drawparallels(np.arange(llcrnrlat,urcrnrlat,dy),labels=[False,False,False,False],dashes=[0.5,0.5]) 
#     lons = m.drawmeridians(np.arange(llcrnrlon,urcrnrlon,dx),labels=[False,False,False,False],dashes=[0.5,0.5]) 
    lats = m.drawparallels(np.reshape(np.linspace(llcrnrlat,urcrnrlat,ny+1),(ny+1,)),labels=[False,False,False,False],dashes=[0.5,0.5]) 
    lons = m.drawmeridians(np.reshape(np.linspace(llcrnrlon,urcrnrlon,nx+1),(nx+1,)),labels=[False,False,False,False],dashes=[0.5,0.5]) 

    lat_lines = chain(*(tup[1][0] for tup in lats.items()))
    lon_lines = chain(*(tup[1][0] for tup in lons.items()))
    all_lines = chain(lat_lines, lon_lines)
    
    # cycle through these lines and set the desired style
    for line in all_lines:
        line.set(linestyle='-', alpha=0.3, color='grey')

    m.drawstates(linewidth=0.5, linestyle='solid', color='k')
#     m.drawrivers(linewidth=1.0, linestyle='solid', color='blue')
    return m

# ===============================================================================
random.seed(40)
root_dir = '/glade/u/home/hongli/work/russian/ens_forc_wrf2'
format_stnlist_file = os.path.join(root_dir,'scripts/step2_prepare_complete_stnlist/stnlist_slope.txt')
gridinfo_file = os.path.join(root_dir,'GMET_tpl/inputs/gridinfo.nc')

asc_dir = os.path.join(root_dir, 'scripts/hmet_ascii_data_1day_lead')
asc_files = [f for f in os.listdir(asc_dir) if '.asc' in f]
asc_files = sorted(asc_files)

outfolder = 'scripts/step4_sample_stnlist_random'
if os.path.exists(os.path.join(root_dir, outfolder)):
    shutil.rmtree(os.path.join(root_dir, outfolder))
os.makedirs(os.path.join(root_dir, outfolder))
ofile_name_base = 'stnlist'
dpi_value = 90

# read the complete stnlist
data = np.loadtxt(format_stnlist_file, skiprows=1, delimiter=',', dtype='str') #STA_ID[0], LAT[1], LON[2], ELEV[3], SLP_N[4], SLP_E[5], STA_NAME[6]
stn_num = len(data)
 
stn_index = np.arange(stn_num)
# choice_perctls = [0.1]
choice_perctls = np.arange(0.1,1.1,0.1) # (start, end, interval)
wrf_choice_nums = [int(round(p*stn_num)) for p in choice_perctls]

# ==========================================================================================
# uniformly sample and save
print('save stnlist.txt')
for i,wrf_choice_num in enumerate(wrf_choice_nums):
    
    choice_perctl = choice_perctls[i]
    ofile = ofile_name_base + '_'+str('%03d' %(choice_perctl*100))+'percent.txt'
    print(ofile)
    
    # uniform choice
    choice_index = np.random.choice(stn_index, size=wrf_choice_num, replace=False)

    # save stnlist.txt        
    f_in = open(format_stnlist_file, 'r') # read in the complete stnlist line-by-line
    content = f_in.readlines()
    
    f_out = open(os.path.join(root_dir, outfolder, ofile), 'w') 
    f_out.write('NSITES\t'+str(wrf_choice_num)+'\n') # total number line
    f_out.write(content[1]) # title line
    for j in choice_index:
        f_out.write(content[j+2]) 

    f_in.close()        
    f_out.close()   

# ==========================================================================================
print('plot distribution')
# lat/lon bounds and central lat/lon
with open(os.path.join(asc_dir, asc_files[0]), 'r') as f:
    content = f.readlines()
    for line in content:
        line = line.strip()        
        if line:
            if ('ncols' in line):
                ncols = int(line.split()[1])
                nx = ncols
            elif ('nrows' in line):
                nrows = int(line.split()[1])
                ny = nrows
            elif ('xllcorner' in line):
                xllcorner = float(line.split()[1])
            elif ('yllcorner' in line):
                yllcorner = float(line.split()[1])
            elif ('cellsize' in line):
                cellsize = float(line.split()[1])

p = Proj(proj='utm',zone=10,ellps='WGS84', preserve_units=False)
start_lon, start_lat = p(xllcorner, yllcorner, inverse=True)
end_lon, end_lat = p(xllcorner+cellsize*nx, yllcorner+cellsize*ny, inverse=True)

lat_0=0.5*(start_lat+end_lat)
lon_0=0.5*(start_lon+end_lon)
                
# plot
stnlist_files = [f for f in os.listdir(os.path.join(root_dir, outfolder)) if ofile_name_base in f]
stnlist_files = sorted(stnlist_files)
data = np.loadtxt(os.path.join(root_dir,outfolder,stnlist_files[-1]), skiprows=1, delimiter=',', dtype='str')
total_stn_num = len(data)

nrow = 3
ncol = 4
fig, ax = plt.subplots(nrow, ncol)
fig.set_figwidth(4*ncol) 
fig.set_figheight(4.5*nrow)

for i in range(nrow):
    for j in range(ncol):
        
        k = i*ncol+j            
        if k<len(stnlist_files):  
            
            # read sampled stnlist.txt
            stnlist_file = os.path.join(root_dir, outfolder, stnlist_files[k])
            data = np.loadtxt(stnlist_file, skiprows=1, delimiter=',', dtype='str') #STA_ID[0], LAT[1], LON[2], ELEV[3], SLP_N[4], SLP_E[5], STA_NAME[6]
            stn_num = len(data)
            stn_lons = [float(data[i][2]) for i in range(stn_num)]
            stn_lats = [float(data[i][1]) for i in range(stn_num)]
            print(str(stn_num) +' Grids')

            m = plot_basemap(llcrnrlon=start_lon,llcrnrlat=start_lat,
                             urcrnrlon=end_lon,urcrnrlat=end_lat, ax=ax[i,j],
                             nx=nx,ny=ny,lat_0=lat_0,lon_0=lon_0) # plot Basemap                           

            x, y = m(stn_lons,stn_lats) # convert the lat/lon values to x/y projections.
            m.plot(x, y, 'bs', markersize=2) # plot sampeld grid points

            # set title
            perctl=round(stn_num/total_stn_num*100,0)
            title_str = '('+chr(ord('a') + k) +') ' + str(stn_num)  +' Sampled Grids ('+str('%d' %(perctl))+'%)'
            ax[i,j].set_title(title_str, fontsize='small', fontweight='semibold')

        else: # blank axis
            ax[i,j].axis('off')

# save plot
fig.tight_layout()
ofile = 'sample_grids_dist.png'
fig.savefig(os.path.join(root_dir, outfolder, ofile), dpi=dpi_value)
plt.close(fig)    

print('Done')

/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


save stnlist.txt
stnlist_010percent.txt
stnlist_020percent.txt
stnlist_030percent.txt
stnlist_040percent.txt
stnlist_050percent.txt
stnlist_060percent.txt
stnlist_070percent.txt
stnlist_080percent.txt
stnlist_090percent.txt
stnlist_100percent.txt
plot distribution
39 Grids


/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/ipykernel_launcher.py:12: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  if sys.path[0] == '':
/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/ipykernel_launcher.py:38: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.


79 Grids
118 Grids
157 Grids
196 Grids
236 Grids
275 Grids
314 Grids
354 Grids
393 Grids
Done
